In [1]:
# General
import os, time, glob, warnings
import numpy as np
import pandas as pd
import datetime as dt
import tables as tb
pd.options.display.max_columns = 24

# Matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Cartopy & geopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.feature import ShapelyFeature
from cartopy.io.shapereader import Reader
from geopy import distance

# Autocomplete
%config Completer.use_jedi = False

In [2]:
import sys
sys.path.append('../src/')
import learner

import importlib
importlib.reload(learner)

l = learner.DeepLearner()

In [68]:
pd.read_csv('../notebooks/variables-stats/madrid_gases.csv').iloc[:-3]

,code,gas,min,mean,median,max,std
0,1,SO2,0.0,8.531437,7.00,199.0,7.399099
1,6,CO,0.0,0.425366,0.30,16.8,0.385224
2,7,NO,0.0,25.808452,7.00,1502.0,54.883410
3,8,NO2,0.0,41.121209,34.00,586.0,31.152513
4,9,PM25,0.0,11.112877,9.00,215.0,8.930240
5,10,PM10,0.0,22.489369,17.00,721.0,21.026431
6,12,NOx,0.0,80.683271,45.00,2755.0,108.019257
7,14,O3,0.0,46.813639,44.89,235.7,32.713721
8,20,TOL,0.1,2.987406,1.60,174.2,4.255430
9,30,BEN,0.0,0.662704,0.30,45.4,0.984969


# Table to compare models

To be able to compare two models, they need to:
- have intersecting variables.
- have intersecting horizons.

In [5]:
def normalize(arr, l, id_idx_v, vc):
    # Compute average value of test set:
    with tb.open_file(l.dataset_d_raw['time_series'], mode='r') as h5_raw:
        avg_t = np.nanmean(h5_raw.root['/v{}'.format(vc)][slice(*l.test_extent), 1:], axis=0)
    return np.nanmedian(arr[:, l.vi_si_map[id_idx_v], id_idx_v] / avg_t, axis=1)

def get_meta(fpath, l, id_idx_v, vc):
    d = dict()
    with tb.open_file(fpath, mode='r') as h5_in:
        d['model_name'] = os.path.basename(os.path.dirname(fpath))
        d['interp'] = h5_in.root._v_attrs['interp']
        d['scal'] = h5_in.root._v_attrs['scaling']
        d['opt'] = str(h5_in.root._v_attrs['optimizer']).split('.')[-1][:-2]
        d['loss'] = h5_in.root._v_attrs['loss']
        d['lr'] = h5_in.root._v_attrs['lr']
        d['epochs'] = h5_in.root._v_attrs['epochs']
        d['model'] = 'ST-AQF' if 'ST' in h5_in.root._v_attrs['name'] else h5_in.root._v_attrs['name'].split('_')[0]
        d['rmse'] = h5_in.root._v_attrs['rmse_mod']
        d['n_x'] = h5_in.root._v_attrs['n_x']
        d['s'] = h5_in.root._v_attrs['shift']
        d['n_y'] = h5_in.root._v_attrs['n_y']
        d['n_v'] = h5_in.root._v_attrs['n_v']
        d['idx_v'] = h5_in.root._v_attrs['idx_v']
        d['calendar_aware'] = h5_in.root._v_attrs['calendar_aware']
        d['weather_aware'] = h5_in.root._v_attrs['weather_aware']
        d['trained_in'] = h5_in.root._v_attrs['trained_in'].split('.')[0]
        d['trained_with'] = h5_in.root._v_attrs['trained_with'][0]
        d['t'] = time.strftime('%H:%M', time.gmtime(h5_in.root._v_attrs['train_duration_s']))
        d['rmse'] = np.nanmedian(h5_in.root._v_attrs['rmse_mod'], axis=1)[:, id_idx_v]
        d['nrmse'] = normalize(h5_in.root._v_attrs['rmse_mod'], l, id_idx_v, vc)
        d['mae'] = np.nanmedian(h5_in.root._v_attrs['mae_mod'], axis=1)[:, id_idx_v]
        return d

def get_idx_v(fpath):
    with tb.open_file(fpath, mode='r') as h5_in:
        idx_v = h5_in.root._v_attrs['idx_v']
        return sorted(idx_v) if idx_v is not None else np.arange(11)

def result_table(model_paths):
    # Preparation 
    columns = ['model_name', 'interp', 'scal', 'opt', 'loss', 'lr', 'epochs',
               'n_x', 's', 'n_y', 'n_v', 'model',
               'calendar_aware', 'weather_aware', 'trained_in', 'trained_with', 't', 'vn', 'vc']
    df = pd.DataFrame()
    r = 0
    # Actual table
    for id_m, mp in enumerate(model_paths):
        idx_v_l = get_idx_v(mp)
        #for vn in os.path.dirname(mp).split('_')[-1].split('-'):
        for id_idx_v, idx_v in enumerate(idx_v_l):
            l = learner.DeepLearner()
            vc, vn = list(l.vc_map.keys())[idx_v], list(l.vn_map.keys())[idx_v]
            l.set_idx_v(idx_v_l)
            d = get_meta(mp, l, id_idx_v, vc)
            d['vn'], d['vc'] = vn, vc
            df.loc[r, columns] = [d[c] for c in columns]
            for err in ['nrmse', 'rmse', 'mae']:
                err_cols = ['{} {}h'.format(err, h) for h in range(d['s'], d['s'] + d['n_y'])]
                df.loc[r, err_cols] = d[err].round(3)
                df.loc[r, err] = d[err].mean().round(3)
            r += 1
    df[['epochs', 'n_x', 's', 'n_y', 'n_v']] = df[['epochs', 'n_x', 's', 'n_y', 'n_v']].astype(int)
    #df.set_index(columns, inplace=True)
    df.to_excel('./tmp/{}-tmp.xlsx'.format(time.strftime('%Y%m%d-%H%M%S', time.gmtime(time.time()))))
    return df

In [75]:
#for e in ['nrmse', 'rmse', 'mae']:
#    for k in l.vn_map.keys():
#        print(k)
#        print(df.loc[df.vn == k, ['interp', 'scal', e]].sort_values(e).iloc[0:2].values)
#    print()

In [74]:
#for e in ['nrmse', 'rmse', 'mae']:
#    for k in l.vn_map.keys():
#        print(k)
#        print(df.loc[(df.vn == k) & (df.lr == 0.001) & (df.opt == 'Adamax') & (df.loss != 'msle'),
#                     ['loss', e]].sort_values(e, ascending=True).iloc[0:5].values)

In [73]:
#for e in ['nrmse', 'rmse', 'mae']:
#    for k in l.vn_map.keys():
#        print(k)
#        #print(df.loc[(df.vn == k) & (df.lr == 0.001) & (df.opt == 'Adamax') & (df.loss != 'msle'),
#        #             ['loss', e]].sort_values(e, ascending=True).iloc[0:5][e].values)
#        mi, ma = df.loc[(df.vn == k) & (df.lr == 0.001) & (df.opt == 'Adamax') & (df.loss != 'msle'),
#                     ['loss', e]].sort_values(e, ascending=True).iloc[0:5][e].values
#        print((ma - mi) /ma * 100)
#        # pct_change
#    print()

## Compare interp x scaling

Albaricoque:

In [5]:
folder = '../models/madrid/map-analysis-again/'
model_paths = sorted([f for f in glob.glob('{}/*/*.h5'.format(folder)) if 'pred_truth' not in f])

df = result_table(model_paths)
df

<ipython-input-4-c7e4de652f97>:4: RuntimeWarning: Mean of empty slice
  avg_t = np.nanmean(h5_raw.root['/v{}'.format(vc)][slice(*l.test_extent), 1:], axis=0)


,model_name,interp,scal,opt,loss,lr,epochs,n_x,s,n_y,n_v,model,...,nrmse 4h,nrmse,rmse 1h,rmse 2h,rmse 3h,rmse 4h,rmse,mae 1h,mae 2h,mae 3h,mae 4h,mae
0,ST-AQF_norm_linear_J0.0_nx04_ny04_sh01_mse_lr0...,linear,norm,Adamax,mse,0.001,100,4,1,4,11,ST-AQF,...,0.534,0.520,4.205,4.230,4.285,4.345,4.266,3.266,3.292,3.323,3.352,3.308
1,ST-AQF_norm_linear_J0.0_nx04_ny04_sh01_mse_lr0...,linear,norm,Adamax,mse,0.001,100,4,1,4,11,ST-AQF,...,1.219,1.180,0.353,0.356,0.361,0.366,0.359,0.323,0.324,0.326,0.329,0.325
2,ST-AQF_norm_linear_J0.0_nx04_ny04_sh01_mse_lr0...,linear,norm,Adamax,mse,0.001,100,4,1,4,11,ST-AQF,...,3.370,3.253,45.993,46.604,47.777,49.149,47.381,43.458,43.982,44.530,44.999,44.242
3,ST-AQF_norm_linear_J0.0_nx04_ny04_sh01_mse_lr0...,linear,norm,Adamax,mse,0.001,100,4,1,4,11,ST-AQF,...,0.798,0.778,27.627,28.083,28.773,29.453,28.484,23.533,23.949,24.396,24.791,24.167
4,ST-AQF_norm_linear_J0.0_nx04_ny04_sh01_mse_lr0...,linear,norm,Adamax,mse,0.001,100,4,1,4,11,ST-AQF,...,0.936,0.916,9.152,9.182,9.316,9.496,9.287,8.163,8.179,8.250,8.379,8.243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,4,1,4,11,ST-AQF,...,1.032,0.913,49.077,50.814,57.844,63.095,55.208,32.856,34.085,36.607,40.187,35.934
62,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,4,1,4,11,ST-AQF,...,0.585,0.569,28.461,28.758,29.445,30.219,29.221,22.588,22.921,23.684,24.515,23.427
63,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,4,1,4,11,ST-AQF,...,1.373,1.345,2.527,2.485,2.488,2.519,2.505,1.521,1.513,1.567,1.635,1.559
64,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,4,1,4,11,ST-AQF,...,1.228,1.200,0.477,0.478,0.489,0.518,0.491,0.327,0.330,0.341,0.356,0.339


Albaricoque:

In [39]:
folder = '../models/madrid/map-analysis-single/'
model_paths = sorted([f for f in glob.glob('{}/*/*.h5'.format(folder)) if 'pred_truth' not in f])

df = result_table(model_paths)
df

<ipython-input-4-c7e4de652f97>:4: RuntimeWarning: Mean of empty slice
  avg_t = np.nanmean(h5_raw.root['/v{}'.format(vc)][slice(*l.test_extent), 1:], axis=0)


,model_name,interp,scal,opt,loss,lr,epochs,n_x,s,n_y,n_v,model,...,nrmse 4h,nrmse,rmse 1h,rmse 2h,rmse 3h,rmse 4h,rmse,mae 1h,mae 2h,mae 3h,mae 4h,mae
0,ST-AQF_norm_linear_J0.0_nx04_ny04_sh01_mse_lr0...,linear,norm,Adamax,mse,0.001,100,4,1,4,1,ST-AQF,...,1.325,1.235,0.510,0.542,0.567,0.587,0.551,0.417,0.442,0.464,0.481,0.451
1,ST-AQF_norm_linear_J0.0_nx04_ny04_sh01_mse_lr0...,linear,norm,Adamax,mse,0.001,100,4,1,4,1,ST-AQF,...,0.870,0.811,0.269,0.274,0.281,0.288,0.278,0.236,0.240,0.247,0.254,0.244
2,ST-AQF_norm_linear_J0.0_nx04_ny04_sh01_mse_lr0...,linear,norm,Adamax,mse,0.001,100,4,1,4,1,ST-AQF,...,2.035,1.970,0.669,0.719,0.756,0.779,0.731,0.391,0.418,0.439,0.452,0.425
3,ST-AQF_norm_linear_J0.0_nx04_ny04_sh01_mse_lr0...,linear,norm,Adamax,mse,0.001,100,4,1,4,1,ST-AQF,...,2.470,2.207,29.572,31.273,34.260,37.115,33.055,26.536,27.919,29.532,30.731,28.680
4,ST-AQF_norm_linear_J0.0_nx04_ny04_sh01_mse_lr0...,linear,norm,Adamax,mse,0.001,100,4,1,4,1,ST-AQF,...,0.693,0.554,15.238,18.100,21.841,24.693,19.968,12.161,14.229,16.800,19.323,15.628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,4,1,4,1,ST-AQF,...,0.430,0.325,11.899,14.424,18.596,22.307,16.806,8.882,10.655,14.033,17.243,12.703
62,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,4,1,4,1,ST-AQF,...,0.600,0.524,7.844,9.124,10.305,11.073,9.587,4.881,5.758,6.654,7.353,6.161
63,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,4,1,4,1,ST-AQF,...,0.582,0.530,4.696,5.505,5.908,6.240,5.587,3.293,3.942,4.276,4.518,4.007
64,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,4,1,4,1,ST-AQF,...,0.312,0.268,1.707,2.037,2.354,2.590,2.172,1.167,1.341,1.545,1.722,1.444


## Compare optimizer x loss x lr

Limonero:

In [6]:
folder = '../models/madrid/compare-opt-loss-lr-again/'
model_paths = sorted([f for f in glob.glob('{}/*/*.h5'.format(folder)) if 'pred_truth' not in f])

df = result_table(model_paths)
df

<ipython-input-4-c7e4de652f97>:4: RuntimeWarning: Mean of empty slice
  avg_t = np.nanmean(h5_raw.root['/v{}'.format(vc)][slice(*l.test_extent), 1:], axis=0)


,model_name,interp,scal,opt,loss,lr,epochs,n_x,s,n_y,n_v,model,...,nrmse 4h,nrmse,rmse 1h,rmse 2h,rmse 3h,rmse 4h,rmse,mae 1h,mae 2h,mae 3h,mae 4h,mae
0,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mae_l...,nearest,stand,Adadelta,mae,0.000100,100,4,1,4,11,ST-AQF,...,0.586,0.581,4.528,4.489,4.495,4.514,4.507,3.320,3.299,3.371,3.301,3.323
1,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mae_l...,nearest,stand,Adadelta,mae,0.000100,100,4,1,4,11,ST-AQF,...,0.730,0.722,0.237,0.232,0.235,0.236,0.235,0.159,0.156,0.159,0.158,0.158
2,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mae_l...,nearest,stand,Adadelta,mae,0.000100,100,4,1,4,11,ST-AQF,...,2.271,2.240,34.436,34.308,34.686,34.027,34.364,20.900,20.725,21.100,20.877,20.900
3,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mae_l...,nearest,stand,Adadelta,mae,0.000100,100,4,1,4,11,ST-AQF,...,0.769,0.767,27.405,27.360,27.578,27.331,27.418,22.055,22.128,22.573,22.215,22.243
4,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mae_l...,nearest,stand,Adadelta,mae,0.000100,100,4,1,4,11,ST-AQF,...,0.777,0.771,7.535,7.641,7.539,7.794,7.627,5.873,5.814,5.710,5.910,5.827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_msle_...,nearest,stand,SGD,msle,0.000005,100,4,1,4,11,ST-AQF,...,1.285,1.272,76.842,74.546,74.896,75.840,75.531,50.127,51.870,50.710,51.227,50.983
1052,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_msle_...,nearest,stand,SGD,msle,0.000005,100,4,1,4,11,ST-AQF,...,0.655,0.653,34.469,34.352,34.273,34.384,34.369,28.859,28.780,28.612,28.754,28.751
1053,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_msle_...,nearest,stand,SGD,msle,0.000005,100,4,1,4,11,ST-AQF,...,1.526,1.534,2.837,2.804,2.767,2.782,2.798,1.918,1.878,1.899,1.892,1.897
1054,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_msle_...,nearest,stand,SGD,msle,0.000005,100,4,1,4,11,ST-AQF,...,1.272,1.277,0.531,0.525,0.528,0.533,0.529,0.385,0.390,0.380,0.388,0.386


Espino:

In [8]:
folder = '../models/madrid/compare-opt-loss-lr-single/'
model_paths = sorted([f for f in glob.glob('{}/*/*.h5'.format(folder)) if 'pred_truth' not in f])

df = result_table(model_paths)
df

<ipython-input-4-c7e4de652f97>:4: RuntimeWarning: Mean of empty slice
  avg_t = np.nanmean(h5_raw.root['/v{}'.format(vc)][slice(*l.test_extent), 1:], axis=0)


,model_name,interp,scal,opt,loss,lr,epochs,n_x,s,n_y,n_v,model,...,nrmse 4h,nrmse,rmse 1h,rmse 2h,rmse 3h,rmse 4h,rmse,mae 1h,mae 2h,mae 3h,mae 4h,mae
0,ST-AQF_stand_linear_J0.0_nx04_ny04_sh01_mae_lr...,linear,stand,Adadelta,mae,0.001,100,4,1,4,1,ST-AQF,...,1.245,1.233,0.515,0.519,0.515,0.520,0.517,0.365,0.372,0.364,0.372,0.368
1,ST-AQF_stand_linear_J0.0_nx04_ny04_sh01_mae_lr...,linear,stand,Adadelta,mae,0.001,100,4,1,4,1,ST-AQF,...,0.664,0.667,0.237,0.236,0.234,0.236,0.236,0.130,0.147,0.135,0.131,0.136
2,ST-AQF_stand_linear_J0.0_nx04_ny04_sh01_mae_lr...,linear,stand,Adadelta,mae,0.001,100,4,1,4,1,ST-AQF,...,2.186,2.099,0.858,0.868,0.852,0.890,0.867,0.394,0.426,0.345,0.430,0.399
3,ST-AQF_stand_linear_J0.0_nx04_ny04_sh01_mae_lr...,linear,stand,Adadelta,mae,0.001,100,4,1,4,1,ST-AQF,...,2.090,2.055,29.112,30.777,32.284,31.549,30.930,12.616,14.602,16.239,14.187,14.411
4,ST-AQF_stand_linear_J0.0_nx04_ny04_sh01_mae_lr...,linear,stand,Adadelta,mae,0.001,100,4,1,4,1,ST-AQF,...,0.750,0.725,25.935,24.967,25.950,26.875,25.931,19.372,18.564,19.542,20.395,19.468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,ST-AQF_stand_linear_J0.0_nx04_ny04_sh01_msle_l...,linear,stand,SGD,msle,0.001,100,4,1,4,1,ST-AQF,...,0.659,0.657,34.642,34.634,34.164,34.638,34.520,28.465,28.490,28.255,28.562,28.443
260,ST-AQF_stand_linear_J0.0_nx04_ny04_sh01_msle_l...,linear,stand,SGD,msle,0.001,100,4,1,4,1,ST-AQF,...,0.873,0.873,15.730,15.761,15.747,15.789,15.757,12.736,12.740,12.769,12.774,12.755
261,ST-AQF_stand_linear_J0.0_nx04_ny04_sh01_msle_l...,linear,stand,SGD,msle,0.001,100,4,1,4,1,ST-AQF,...,0.810,0.815,8.215,8.175,8.119,8.201,8.178,6.735,6.642,6.525,6.594,6.624
262,ST-AQF_stand_linear_J0.0_nx04_ny04_sh01_msle_l...,linear,stand,SGD,msle,0.001,100,4,1,4,1,ST-AQF,...,0.555,0.553,4.277,4.267,4.284,4.274,4.276,3.377,3.357,3.380,3.363,3.369


## Compare equivalent models

Espino:

In [298]:
model_paths = [
    '../models/madrid/experiment-n_x-update/ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_lr0.001_Adamax_ep100_CO/ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_lr0.001_Adamax_ep100_CO.h5',
    '../models/madrid/experiment-n_x/ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_lr0.001_Adamax_ep100_CO/ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_lr0.001_Adamax_ep100_CO.h5'
]

df = result_table(model_paths)
df

,model_name,interp,scal,opt,loss,lr,epochs,n_x,s,n_y,n_v,model,...,nrmse 4h,nrmse,rmse 1h,rmse 2h,rmse 3h,rmse 4h,rmse,mae 1h,mae 2h,mae 3h,mae 4h,mae
0,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,4,1,4,1,ST-AQF,...,0.565,0.456,0.120,0.139,0.164,0.184,0.152,0.074,0.083,0.100,0.115,0.093
1,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,4,1,4,1,ST-AQF,...,0.560,0.459,0.121,0.141,0.166,0.186,0.153,0.075,0.090,0.107,0.120,0.098


Espino:

In [299]:
model_paths = [
    '../models/madrid/experiment-n_x-update/ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_lr0.001_Adamax_ep100_NO2/ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_lr0.001_Adamax_ep100_NO2.h5',
    '../models/madrid/experiment-n_x/ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_lr0.001_Adamax_ep100_NO2/ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_lr0.001_Adamax_ep100_NO2.h5'
]

df = result_table(model_paths)
df

,model_name,interp,scal,opt,loss,lr,epochs,n_x,s,n_y,n_v,model,...,nrmse 4h,nrmse,rmse 1h,rmse 2h,rmse 3h,rmse 4h,rmse,mae 1h,mae 2h,mae 3h,mae 4h,mae
0,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,4,1,4,1,ST-AQF,...,0.634,0.485,12.332,15.618,19.800,22.993,17.686,9.080,11.113,13.939,16.570,12.676
1,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,4,1,4,1,ST-AQF,...,0.632,0.484,12.286,15.715,19.683,22.816,17.625,9.161,11.020,13.938,16.625,12.686


Espino:

In [300]:
model_paths = [
    '../models/madrid/experiment-n_x-update/ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_lr0.001_Adamax_ep100_all/ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_lr0.001_Adamax_ep100_all.h5',
    #'../models/madrid/experiment-n_x/ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_lr0.001_Adamax_ep100_al'
]

df = result_table(model_paths)
df

<ipython-input-297-a41110d76d92>:4: RuntimeWarning: Mean of empty slice
  avg_t = np.nanmean(h5_raw.root['/v{}'.format(vc)][slice(*l.test_extent), 1:], axis=0)


,model_name,interp,scal,opt,loss,lr,epochs,n_x,s,n_y,n_v,model,...,nrmse 4h,nrmse,rmse 1h,rmse 2h,rmse 3h,rmse 4h,rmse,mae 1h,mae 2h,mae 3h,mae 4h,mae
0,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,4,1,4,11,ST-AQF,...,0.524,0.512,4.043,4.035,4.099,4.191,4.092,3.088,3.094,3.136,3.180,3.124
1,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,4,1,4,11,ST-AQF,...,0.600,0.554,0.168,0.172,0.185,0.201,0.182,0.114,0.116,0.123,0.129,0.120
2,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,4,1,4,11,ST-AQF,...,1.919,1.770,26.038,26.640,28.841,30.612,28.032,17.537,17.287,17.546,17.900,17.567
3,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,4,1,4,11,ST-AQF,...,0.664,0.586,18.438,19.753,21.710,23.636,20.884,13.867,14.725,16.263,17.430,15.571
4,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,4,1,4,11,ST-AQF,...,0.660,0.647,6.418,6.442,6.509,6.680,6.512,4.794,4.772,4.912,5.109,4.897
5,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,4,1,4,11,ST-AQF,...,0.730,0.715,12.516,12.473,12.739,13.139,12.717,9.244,9.142,9.281,9.482,9.287
6,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,4,1,4,11,ST-AQF,...,1.049,0.930,49.508,52.095,58.302,63.994,55.975,34.691,34.899,37.828,40.169,36.897
7,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,4,1,4,11,ST-AQF,...,0.588,0.568,28.643,28.879,29.600,30.417,29.385,22.718,23.018,23.787,24.648,23.543
8,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,4,1,4,11,ST-AQF,...,1.372,1.345,2.546,2.506,2.511,2.542,2.526,1.537,1.522,1.551,1.617,1.557
9,ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,4,1,4,11,ST-AQF,...,1.234,1.201,0.495,0.484,0.499,0.526,0.501,0.340,0.337,0.347,0.361,0.346


## Extensibility

Espino:

In [301]:
folder = '../models/madrid/experiment-n_x/'
model_paths = sorted([f for f in glob.glob('{}/*/*.h5'.format(folder)) if 'pred_truth' not in f])

df = result_table(model_paths)
df

<ipython-input-297-a41110d76d92>:4: RuntimeWarning: Mean of empty slice
  avg_t = np.nanmean(h5_raw.root['/v{}'.format(vc)][slice(*l.test_extent), 1:], axis=0)


,model_name,interp,scal,opt,loss,lr,epochs,n_x,s,n_y,n_v,model,...,nrmse 4h,nrmse,rmse 1h,rmse 2h,rmse 3h,rmse 4h,rmse,mae 1h,mae 2h,mae 3h,mae 4h,mae
0,ST-AQF_stand_nearest_J0.0_nx01_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,1,1,4,1,ST-AQF,...,1.034,0.894,0.312,0.383,0.424,0.450,0.392,0.155,0.209,0.251,0.280,0.224
1,ST-AQF_stand_nearest_J0.0_nx01_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,1,1,4,1,ST-AQF,...,0.592,0.486,0.121,0.151,0.176,0.194,0.160,0.072,0.091,0.110,0.123,0.099
2,ST-AQF_stand_nearest_J0.0_nx01_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,1,1,4,1,ST-AQF,...,1.811,1.692,0.586,0.665,0.719,0.754,0.681,0.165,0.230,0.279,0.315,0.247
3,ST-AQF_stand_nearest_J0.0_nx01_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,1,1,4,1,ST-AQF,...,1.919,1.549,16.921,22.152,26.727,29.845,23.911,8.606,11.467,13.879,16.013,12.491
4,ST-AQF_stand_nearest_J0.0_nx01_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,1,1,4,1,ST-AQF,...,0.676,0.525,12.292,17.432,21.547,24.243,18.879,9.143,12.612,15.716,17.958,13.857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,ST-AQF_stand_nearest_J0.0_nx24_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,24,1,4,1,ST-AQF,...,0.366,0.298,13.647,13.599,15.984,19.074,15.576,10.398,10.196,11.988,14.554,11.784
95,ST-AQF_stand_nearest_J0.0_nx24_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,24,1,4,1,ST-AQF,...,0.564,0.510,8.669,9.112,9.884,10.702,9.592,5.847,6.069,6.478,7.029,6.356
96,ST-AQF_stand_nearest_J0.0_nx24_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,24,1,4,1,ST-AQF,...,0.566,0.516,4.659,5.303,5.662,5.962,5.396,3.258,3.771,4.020,4.217,3.817
97,ST-AQF_stand_nearest_J0.0_nx24_ny04_sh01_mse_l...,nearest,stand,Adamax,mse,0.001,100,24,1,4,1,ST-AQF,...,0.307,0.263,1.717,1.956,2.215,2.448,2.084,1.190,1.337,1.481,1.599,1.402


Espino:

In [302]:
folder = '../models/madrid/experiment-extensibility/'
model_paths = sorted([f for f in glob.glob('{}/*/*.h5'.format(folder)) if 'pred_truth' not in f])

df = result_table(model_paths)
df.set_index(df.columns[:18].tolist())

<ipython-input-297-a41110d76d92>:4: RuntimeWarning: Mean of empty slice
  avg_t = np.nanmean(h5_raw.root['/v{}'.format(vc)][slice(*l.test_extent), 1:], axis=0)


vc  \
model_name                                         interp  scal  opt    loss lr    epochs n_x s n_y n_v model  calendar_aware weather_aware trained_in trained_with               t     vn         
ST-AQF_stand_nearest_J0.0_nx01_ny04_sh01_mse_lr... nearest stand Adamax mse  0.001 100    1   1 4   8   ST-AQF False          False         espino     NVIDIA GeForce GTX TITAN X 01:11 SO2   01   
                                                                                                                                                                                        CO    06   
                                                                                                                                                                                        NO    07   
                                                                                                                                                                                        NO2   08   
                                                                                                                                                                                        PM25  09   
...                                                                                                                                                                                           ..   
ST-AQF_stand_nearest_J0.0_nx24_ny04_sh01_mse_lr... nearest stand Adamax mse  0.001 100    24  1 4   6   ST-AQF False          False         espino     NVIDIA GeForce GTX TITAN X 03:16 CO    06   
                                                                                                                                                                                        NO    07   
                                                                                                                                                                                        NO2   08   
                                                                                                                                                                                        PM25  09   
                                                                                                                                                                                        PM10  10   

                                                                                                                                                                                              nrmse 1h  \
model_name                                         interp  scal  opt    loss lr    epochs n_x s n_y n_v model  calendar_aware weather_aware trained_in trained_with               t     vn               
ST-AQF_stand_nearest_J0.0_nx01_ny04_sh01_mse_lr... nearest stand Adamax mse  0.001 100    1   1 4   8   ST-AQF False          False         espino     NVIDIA GeForce GTX TITAN X 01:11 SO2      0.478   
                                                                                                                                                                                        CO       0.496   
                                                                                                                                                                                        NO       1.574   
                                                                                                                                                                                        NO2      0.582   
                                                                                                                                                                                        PM25     0.654   
...                                                                                                                                                                                                ...   
ST-AQF_stand_nearest_J0.0

Limonero:

In [9]:
folder = '../models/madrid/experiment-extensibility/'
model_paths = sorted([f for f in glob.glob('{}/*/*.h5'.format(folder)) if 'pred_truth' not in f])

df = result_table(model_paths)
df.set_index(df.columns[:18].tolist())

vc  \
model_name                                         interp  scal  opt    loss lr    epochs n_x s n_y n_v model  calendar_aware weather_aware trained_in trained_with               t     vn         
ST-AQF_stand_nearest_J0.0_nx01_ny04_sh01_mse_lr... nearest stand Adamax mse  0.001 100    1   1 4   2   ST-AQF True           False         limonero   NVIDIA GeForce GTX TITAN X 00:44 NO2   08   
                                                                                                                                                                                        PM10  10   
ST-AQF_stand_nearest_J0.0_nx01_ny04_sh01_mse_lr... nearest stand Adamax mse  0.001 100    1   1 4   4   ST-AQF True           False         limonero   NVIDIA GeForce RTX 2070    00:28 SO2   01   
                                                                                                                                                                                        CO    06   
                                                                                                                                                                                        NO2   08   
                                                                                                                                                                                        PM10  10   
ST-AQF_stand_nearest_J0.0_nx02_ny04_sh01_mse_lr... nearest stand Adamax mse  0.001 100    2   1 4   2   ST-AQF True           False         limonero   NVIDIA GeForce GTX TITAN X 00:49 NO2   08   
                                                                                                                                                                                        PM10  10   
ST-AQF_stand_nearest_J0.0_nx02_ny04_sh01_mse_lr... nearest stand Adamax mse  0.001 100    2   1 4   4   ST-AQF True           False         limonero   NVIDIA GeForce RTX 2070    00:30 SO2   01   
                                                                                                                                                                                        CO    06   
                                                                                                                                                                                        NO2   08   
                                                                                                                                                                                        PM10  10   
ST-AQF_stand_nearest_J0.0_nx03_ny04_sh01_mse_lr... nearest stand Adamax mse  0.001 100    3   1 4   2   ST-AQF True           False         limonero   NVIDIA GeForce GTX TITAN X 00:56 NO2   08   
                                                                                                                                                                                        PM10  10   
ST-AQF_stand_nearest_J0.0_nx03_ny04_sh01_mse_lr... nearest stand Adamax mse  0.001 100    3   1 4   4   ST-AQF True           False         limonero   NVIDIA GeForce RTX 2070    00:58 SO2   01   
                                                                                                                                                                                        CO    06   
                                                                                                                                                                                        NO2   08   
                                                                                                                                                                                        PM10  10   
ST-AQF_stand_nearest_J0.0_nx04_ny04_sh01_mse_lr... nearest stand Adamax mse  0.001 100    4   1 4   2   ST-AQF True           False         limonero   NVIDIA GeForce GTX TITAN X 01:09 NO2   08   
                                                                          

Albaricoque:

In [6]:
folder = '../models/madrid/experiment-n_x/'
model_paths = sorted([f for f in glob.glob('{}/*/*.h5'.format(folder)) if 'pred_truth' not in f])

df = result_table(model_paths)
df.set_index(df.columns[:18].tolist())

<ipython-input-5-c7e4de652f97>:4: RuntimeWarning: Mean of empty slice
  avg_t = np.nanmean(h5_raw.root['/v{}'.format(vc)][slice(*l.test_extent), 1:], axis=0)


,,,,,,,,,,,,,,,,,,vc,nrmse 5h,nrmse 6h,nrmse 7h,nrmse 8h,nrmse,rmse 5h,rmse 6h,rmse 7h,rmse 8h,rmse,mae 5h,mae 6h,mae 7h,mae 8h,mae
model_name,interp,scal,opt,loss,lr,epochs,n_x,s,n_y,n_v,model,calendar_aware,weather_aware,trained_in,trained_with,t,vn,,,,,,,,,,,,,,,,
ST-AQF_stand_nearest_J0.0_nx04_ny04_sh05_mse_lr0.001_Adamax_ep100_BEN,nearest,stand,Adamax,mse,0.001,100,4,5,4,1,ST-AQF,False,False,albaricoque,GeForce GTX TITAN,01:02,BEN,30,1.075,1.050,1.013,0.984,1.031,0.469,0.460,0.450,0.444,0.456,0.292,0.286,0.279,0.276,0.283
ST-AQF_stand_nearest_J0.0_nx04_ny04_sh05_mse_lr0.001_Adamax_ep100_CO,nearest,stand,Adamax,mse,0.001,100,4,5,4,1,ST-AQF,False,False,albaricoque,GeForce GTX TITAN,00:11,CO,06,0.629,0.615,0.615,0.625,0.621,0.210,0.209,0.207,0.208,0.209,0.143,0.143,0.141,0.144,0.143
ST-AQF_stand_nearest_J0.0_nx04_ny04_sh05_mse_lr0.001_Adamax_ep100_EBE,nearest,stand,Adamax,mse,0.001,100,4,5,4,1,ST-AQF,False,False,albaricoque,GeForce GTX TITAN,00:38,EBE,35,1.878,1.889,1.888,1.893,1.887,0.742,0.750,0.758,0.760,0.753,0.344,0.347,0.352,0.349,0.348
ST-AQF_stand_nearest_J0.0_nx04_ny04_sh05_mse_lr0.001_Adamax_ep100_NO,nearest,stand,Adamax,mse,0.001,100,4,5,4,1,ST-AQF,False,False,albaricoque,GeForce GTX TITAN,01:01,NO,07,1.895,1.860,1.859,1.885,1.875,28.398,27.939,28.031,28.892,28.315,17.080,16.941,17.177,17.613,17.203
ST-AQF_stand_nearest_J0.0_nx04_ny04_sh05_mse_lr0.001_Adamax_ep100_NO2,nearest,stand,Adamax,mse,0.001,100,4,5,4,1,ST-AQF,False,False,albaricoque,GeForce GTX TITAN,01:01,NO2,08,0.664,0.656,0.654,0.662,0.659,23.815,23.486,23.355,23.680,23.584,18.394,17.942,17.753,18.056,18.036
ST-AQF_stand_nearest_J0.0_nx04_ny04_sh05_mse_lr0.001_Adamax_ep100_NOx,nearest,stand,Adamax,mse,0.001,100,4,5,4,1,ST-AQF,False,False,albaricoque,GeForce GTX TITAN,01:01,NOx,12,1.054,1.046,1.052,1.081,1.058,61.206,60.631,61.069,63.098,61.501,39.250,38.631,38.417,39.249,38.887
ST-AQF_stand_nearest_J0.0_nx04_ny04_sh05_mse_lr0.001_Adamax_ep100_O3,nearest,stand,Adamax,mse,0.001,100,4,5,4,1,ST-AQF,False,False,albaricoque,GeForce GTX TITAN,01:02,O3,14,0.533,0.534,0.540,0.547,0.539,26.788,26.994,27.576,28.141,27.375,21.731,21.825,22.157,22.444,22.039
ST-AQF_stand_nearest_J0.0_nx04_ny04_sh05_mse_lr0.001_Adamax_ep100_PM10,nearest,stand,Adamax,mse,0.001,100,4,5,4,1,ST-AQF,False,False,albaricoque,GeForce GTX TITAN,01:02,PM10,10,0.668,0.681,0.702,0.702,0.688,12.360,12.594,12.732,12.891,12.644,8.276,8.462,8.565,8.608,8.478
ST-AQF_stand_nearest_J0.0_nx04_ny04_sh05_mse_lr0.001_Adamax_ep100_PM25,nearest,stand,Adamax,mse,0.001,100,4,5,4,1,ST-AQF,False,False,albaricoque,GeForce GTX TITAN,00:08,PM25,09,0.649,0.649,0.657,0.675,0.657,6.614,6.767,6.804,6.888,6.768,4.867,5.010,5.009,5.076,4.990


## Integrability

Espino:

In [305]:
folder = '../models/madrid/experiment-integrability/'
model_paths = sorted([f for f in glob.glob('{}/*/*.h5'.format(folder)) if 'pred_truth' not in f])

df = result_table(model_paths)
df.set_index(df.columns[:18].tolist())

<ipython-input-297-a41110d76d92>:4: RuntimeWarning: Mean of empty slice
  avg_t = np.nanmean(h5_raw.root['/v{}'.format(vc)][slice(*l.test_extent), 1:], axis=0)


vc  \
model_name                                         interp  scal  opt    loss lr    epochs n_x s n_y n_v model  calendar_aware weather_aware trained_in trained_with               t     vn         
ST-AQF-star_stand_nearest_J0.0_nx01_ny04_sh01_m... nearest stand Adamax mse  0.001 100    1   1 4   11  ST-AQF True           True          espino     NVIDIA GeForce GTX TITAN X 01:41 SO2   01   
                                                                                                                                                                                        CO    06   
                                                                                                                                                                                        NO    07   
                                                                                                                                                                                        NO2   08   
                                                                                                                                                                                        PM25  09   
...                                                                                                                                                                                           ..   
ST-AQF-star_stand_nearest_J0.0_nx24_ny04_sh01_m... nearest stand Adamax mse  0.001 100    24  1 4   11  ST-AQF True           True          espino     NVIDIA GeForce GTX TITAN X 09:08 NOx   12   
                                                                                                                                                                                        O3    14   
                                                                                                                                                                                        TOL   20   
                                                                                                                                                                                        BEN   30   
                                                                                                                                                                                        EBE   35   

                                                                                                                                                                                              nrmse 1h  \
model_name                                         interp  scal  opt    loss lr    epochs n_x s n_y n_v model  calendar_aware weather_aware trained_in trained_with               t     vn               
ST-AQF-star_stand_nearest_J0.0_nx01_ny04_sh01_m... nearest stand Adamax mse  0.001 100    1   1 4   11  ST-AQF True           True          espino     NVIDIA GeForce GTX TITAN X 01:41 SO2      0.500   
                                                                                                                                                                                        CO       0.546   
                                                                                                                                                                                        NO       1.936   
                                                                                                                                                                                        NO2      0.525   
                                                                                                                                                                                        PM25     0.693   
...                                                                                                                                                                                                ...   
ST-AQF-star_stand_nearest

Limonero:

In [7]:
folder = '../models/madrid/experiment-n_x/'
model_paths = sorted([f for f in glob.glob('{}/*/*.h5'.format(folder)) if 'pred_truth' not in f])

df = result_table(model_paths)
df.set_index(df.columns[:18].tolist())

<ipython-input-5-c7e4de652f97>:4: RuntimeWarning: Mean of empty slice
  avg_t = np.nanmean(h5_raw.root['/v{}'.format(vc)][slice(*l.test_extent), 1:], axis=0)


vc  \
model_name                                         interp  scal  opt    loss lr    epochs n_x s n_y n_v model  calendar_aware weather_aware trained_in trained_with               t     vn         
ST-AQF_stand_nearest_J0.0_nx01_ny04_sh01_mse_lr... nearest stand Adamax mse  0.001 100    1   1 4   11  ST-AQF False          False         limonero   NVIDIA GeForce RTX 2080 Ti 00:53 SO2   01   
                                                                                                                                                                                        CO    06   
                                                                                                                                                                                        NO    07   
                                                                                                                                                                                        NO2   08   
                                                                                                                                                                                        PM25  09   
...                                                                                                                                                                                           ..   
ST-AQF_stand_nearest_J0.0_nx24_ny04_sh01_mse_lr... nearest stand Adamax mse  0.001 100    24  1 4   11  ST-AQF False          False         limonero   NVIDIA GeForce GTX TITAN X 05:49 NOx   12   
                                                                                                                                                                                        O3    14   
                                                                                                                                                                                        TOL   20   
                                                                                                                                                                                        BEN   30   
                                                                                                                                                                                        EBE   35   

                                                                                                                                                                                              nrmse 1h  \
model_name                                         interp  scal  opt    loss lr    epochs n_x s n_y n_v model  calendar_aware weather_aware trained_in trained_with               t     vn               
ST-AQF_stand_nearest_J0.0_nx01_ny04_sh01_mse_lr... nearest stand Adamax mse  0.001 100    1   1 4   11  ST-AQF False          False         limonero   NVIDIA GeForce RTX 2080 Ti 00:53 SO2      0.482   
                                                                                                                                                                                        CO       0.523   
                                                                                                                                                                                        NO       1.774   
                                                                                                                                                                                        NO2      0.481   
                                                                                                                                                                                        PM25     0.670   
...                                                                                                                                                                                                ...   
ST-AQF_stand_nearest_J0.0

Limonero:

In [6]:
folder = '../models/madrid/experiment-integrability/'
model_paths = sorted([f for f in glob.glob('{}/*/*.h5'.format(folder)) if 'pred_truth' not in f])

df = result_table(model_paths)
df.set_index(df.columns[:18].tolist())

<ipython-input-5-c7e4de652f97>:4: RuntimeWarning: Mean of empty slice
  avg_t = np.nanmean(h5_raw.root['/v{}'.format(vc)][slice(*l.test_extent), 1:], axis=0)


vc  \
model_name                                         interp  scal  opt    loss lr    epochs n_x s n_y n_v model  calendar_aware weather_aware trained_in trained_with               t     vn         
ST-AQF-t_stand_nearest_J0.0_nx01_ny04_sh01_mse_... nearest stand Adamax mse  0.001 100    1   1 4   11  ST-AQF True           False         limonero   NVIDIA GeForce RTX 2080 Ti 00:49 SO2   01   
                                                                                                                                                                                        CO    06   
                                                                                                                                                                                        NO    07   
                                                                                                                                                                                        NO2   08   
                                                                                                                                                                                        PM25  09   
...                                                                                                                                                                                           ..   
ST-AQF-w_stand_nearest_J0.0_nx24_ny04_sh01_mse_... nearest stand Adamax mse  0.001 100    24  1 4   11  ST-AQF False          True          limonero   NVIDIA GeForce RTX 2070    04:32 NOx   12   
                                                                                                                                                                                        O3    14   
                                                                                                                                                                                        TOL   20   
                                                                                                                                                                                        BEN   30   
                                                                                                                                                                                        EBE   35   

                                                                                                                                                                                              nrmse 1h  \
model_name                                         interp  scal  opt    loss lr    epochs n_x s n_y n_v model  calendar_aware weather_aware trained_in trained_with               t     vn               
ST-AQF-t_stand_nearest_J0.0_nx01_ny04_sh01_mse_... nearest stand Adamax mse  0.001 100    1   1 4   11  ST-AQF True           False         limonero   NVIDIA GeForce RTX 2080 Ti 00:49 SO2      0.488   
                                                                                                                                                                                        CO       0.521   
                                                                                                                                                                                        NO       1.877   
                                                                                                                                                                                        NO2      0.483   
                                                                                                                                                                                        PM25     0.675   
...                                                                                                                                                                                                ...   
ST-AQF-w_stand_nearest_J0

## Baselines

Limonero:

In [72]:
folder = '../models/madrid/baselines/'
model_paths = sorted([f for f in glob.glob('{}/*stm*sh01*/*.h5'.format(folder)) if 'pred_truth' not in f])

df = result_table(model_paths)
df.set_index(df.columns[:18].tolist())

<ipython-input-4-c7e4de652f97>:4: RuntimeWarning: Mean of empty slice
  avg_t = np.nanmean(h5_raw.root['/v{}'.format(vc)][slice(*l.test_extent), 1:], axis=0)


,,,,,,,,,,,,,,,,,,vc,nrmse 1h,nrmse 2h,nrmse 3h,nrmse 4h,nrmse,rmse 1h,rmse 2h,rmse 3h,rmse 4h,rmse,mae 1h,mae 2h,mae 3h,mae 4h,mae
model_name,interp,scal,opt,loss,lr,epochs,n_x,s,n_y,n_v,model,calendar_aware,weather_aware,trained_in,trained_with,t,vn,,,,,,,,,,,,,,,,
biLstm_stand_nearest_J0.0_nx04_ny04_sh01_mse_lr0.001_Adamax_ep100_BEN,nearest,stand,Adamax,mse,0.001,100,4,1,4,1,biLstm,False,False,limonero,NVIDIA GeForce RTX 2080 Ti,00:17,BEN,30,0.692,0.849,0.958,1.009,0.877,0.316,0.379,0.417,0.440,0.388,0.172,0.197,0.217,0.232,0.204
biLstm_stand_nearest_J0.0_nx04_ny04_sh01_mse_lr0.001_Adamax_ep100_CO,nearest,stand,Adamax,mse,0.001,100,4,1,4,1,biLstm,False,False,limonero,NVIDIA GeForce RTX 2080 Ti,00:19,CO,06,0.365,0.458,0.543,0.602,0.492,0.127,0.155,0.176,0.193,0.163,0.080,0.102,0.120,0.127,0.107
biLstm_stand_nearest_J0.0_nx04_ny04_sh01_mse_lr0.001_Adamax_ep100_EBE,nearest,stand,Adamax,mse,0.001,100,4,1,4,1,biLstm,False,False,limonero,NVIDIA GeForce RTX 2080 Ti,00:22,EBE,35,1.641,1.769,1.835,1.891,1.784,0.624,0.691,0.732,0.757,0.701,0.219,0.266,0.299,0.324,0.277
biLstm_stand_nearest_J0.0_nx04_ny04_sh01_mse_lr0.001_Adamax_ep100_NO,nearest,stand,Adamax,mse,0.001,100,4,1,4,1,biLstm,False,False,limonero,NVIDIA GeForce RTX 2080 Ti,00:22,NO,07,1.142,1.396,1.576,1.695,1.452,17.248,21.601,25.033,26.963,22.711,8.765,11.631,13.794,15.279,12.367
biLstm_stand_nearest_J0.0_nx04_ny04_sh01_mse_lr0.001_Adamax_ep100_NO2,nearest,stand,Adamax,mse,0.001,100,4,1,4,1,biLstm,False,False,limonero,NVIDIA GeForce RTX 2080 Ti,00:21,NO2,08,0.342,0.456,0.546,0.613,0.489,12.768,16.687,19.694,21.936,17.771,9.151,11.799,14.630,16.474,13.013
biLstm_stand_nearest_J0.0_nx04_ny04_sh01_mse_lr0.001_Adamax_ep100_NOx,nearest,stand,Adamax,mse,0.001,100,4,1,4,1,biLstm,False,False,limonero,NVIDIA GeForce RTX 2080 Ti,00:21,NOx,12,0.595,0.750,0.873,0.965,0.796,34.385,43.571,52.503,58.405,47.216,20.383,25.491,33.388,38.356,29.404
biLstm_stand_nearest_J0.0_nx04_ny04_sh01_mse_lr0.001_Adamax_ep100_O3,nearest,stand,Adamax,mse,0.001,100,4,1,4,1,biLstm,False,False,limonero,NVIDIA GeForce RTX 2080 Ti,00:12,O3,14,0.217,0.304,0.379,0.439,0.335,11.743,16.259,20.091,22.982,17.769,9.006,12.352,15.745,18.343,13.861
biLstm_stand_nearest_J0.0_nx04_ny04_sh01_mse_lr0.001_Adamax_ep100_PM10,nearest,stand,Adamax,mse,0.001,100,4,1,4,1,biLstm,False,False,limonero,NVIDIA GeForce RTX 2080 Ti,00:17,PM10,10,0.475,0.534,0.587,0.625,0.555,8.187,9.581,10.806,11.653,10.057,5.390,6.155,7.039,7.668,6.563
biLstm_stand_nearest_J0.0_nx04_ny04_sh01_mse_lr0.001_Adamax_ep100_PM25,nearest,stand,Adamax,mse,0.001,100,4,1,4,1,biLstm,False,False,limonero,NVIDIA GeForce RTX 2080 Ti,00:11,PM25,09,0.464,0.533,0.564,0.601,0.540,4.704,5.565,6.034,6.321,5.656,3.323,3.916,4.269,4.511,4.004


In [62]:
folder = '../models/madrid/baselines/'
model_paths = sorted([f for f in glob.glob('{}/*sh04*/*.h5'.format(folder)) if 'pred_truth' not in f])

df = result_table(model_paths)
df.set_index(df.columns[:18].tolist())

<ipython-input-4-c7e4de652f97>:4: RuntimeWarning: Mean of empty slice
  avg_t = np.nanmean(h5_raw.root['/v{}'.format(vc)][slice(*l.test_extent), 1:], axis=0)


,,,,,,,,,,,,,,,,,,vc,nrmse 4h,nrmse 5h,nrmse 6h,nrmse 7h,nrmse,rmse 4h,rmse 5h,rmse 6h,rmse 7h,rmse,mae 4h,mae 5h,mae 6h,mae 7h,mae
model_name,interp,scal,opt,loss,lr,epochs,n_x,s,n_y,n_v,model,calendar_aware,weather_aware,trained_in,trained_with,t,vn,,,,,,,,,,,,,,,,
biLstm_stand_nearest_J0.0_nx04_ny04_sh04_mse_lr0.001_Adamax_ep100_BEN,nearest,stand,Adamax,mse,0.001,100,4,4,4,1,biLstm,False,False,limonero,NVIDIA GeForce GTX TITAN X,00:05,BEN,30,1.028,1.042,1.038,1.010,1.030,0.448,0.454,0.455,0.450,0.452,0.250,0.252,0.256,0.254,0.253
biLstm_stand_nearest_J0.0_nx04_ny04_sh04_mse_lr0.001_Adamax_ep100_CO,nearest,stand,Adamax,mse,0.001,100,4,4,4,1,biLstm,False,False,limonero,NVIDIA GeForce GTX TITAN X,00:12,CO,06,0.631,0.655,0.656,0.656,0.650,0.203,0.210,0.213,0.211,0.209,0.134,0.138,0.141,0.141,0.138
biLstm_stand_nearest_J0.0_nx04_ny04_sh04_mse_lr0.001_Adamax_ep100_EBE,nearest,stand,Adamax,mse,0.001,100,4,4,4,1,biLstm,False,False,limonero,NVIDIA GeForce GTX TITAN X,00:09,EBE,35,1.912,1.929,1.938,1.944,1.931,0.767,0.780,0.786,0.787,0.780,0.324,0.339,0.347,0.353,0.341
biLstm_stand_nearest_J0.0_nx04_ny04_sh04_mse_lr0.001_Adamax_ep100_NO,nearest,stand,Adamax,mse,0.001,100,4,4,4,1,biLstm,False,False,limonero,NVIDIA GeForce GTX TITAN X,00:22,NO,07,1.719,1.747,1.768,1.782,1.754,27.471,27.794,27.802,27.889,27.739,15.876,16.344,16.345,16.288,16.213
biLstm_stand_nearest_J0.0_nx04_ny04_sh04_mse_lr0.001_Adamax_ep100_NO2,nearest,stand,Adamax,mse,0.001,100,4,4,4,1,biLstm,False,False,limonero,NVIDIA GeForce GTX TITAN X,00:21,NO2,08,0.613,0.635,0.645,0.648,0.635,22.074,22.843,23.045,23.214,22.794,16.083,16.829,17.226,17.413,16.888
biLstm_stand_nearest_J0.0_nx04_ny04_sh04_mse_lr0.001_Adamax_ep100_NOx,nearest,stand,Adamax,mse,0.001,100,4,4,4,1,biLstm,False,False,limonero,NVIDIA GeForce GTX TITAN X,00:22,NOx,12,0.975,1.001,1.013,1.021,1.003,57.785,59.019,59.749,60.490,59.261,36.879,38.044,38.526,38.423,37.968
biLstm_stand_nearest_J0.0_nx04_ny04_sh04_mse_lr0.001_Adamax_ep100_O3,nearest,stand,Adamax,mse,0.001,100,4,4,4,1,biLstm,False,False,limonero,NVIDIA GeForce GTX TITAN X,00:13,O3,14,0.447,0.483,0.510,0.526,0.491,23.398,25.344,26.606,27.506,25.714,18.526,20.069,21.187,21.825,20.402
biLstm_stand_nearest_J0.0_nx04_ny04_sh04_mse_lr0.001_Adamax_ep100_PM10,nearest,stand,Adamax,mse,0.001,100,4,4,4,1,biLstm,False,False,limonero,NVIDIA GeForce GTX TITAN X,00:13,PM10,10,0.641,0.674,0.699,0.712,0.682,11.885,12.345,12.650,12.774,12.413,7.944,8.339,8.578,8.677,8.384
biLstm_stand_nearest_J0.0_nx04_ny04_sh04_mse_lr0.001_Adamax_ep100_PM25,nearest,stand,Adamax,mse,0.001,100,4,4,4,1,biLstm,False,False,limonero,NVIDIA GeForce GTX TITAN X,00:10,PM25,09,0.614,0.639,0.655,0.665,0.643,6.422,6.605,6.718,6.767,6.628,4.596,4.762,4.897,4.971,4.806


In [60]:
folder = '../models/madrid/baselines/'
model_paths = sorted([f for f in glob.glob('{}/*sh05*/*.h5'.format(folder)) if 'pred_truth' not in f])

df = result_table(model_paths)
df.set_index(df.columns[:18].tolist())

<ipython-input-4-c7e4de652f97>:4: RuntimeWarning: Mean of empty slice
  avg_t = np.nanmean(h5_raw.root['/v{}'.format(vc)][slice(*l.test_extent), 1:], axis=0)


,,,,,,,,,,,,,,,,,,vc,nrmse 5h,nrmse 6h,nrmse 7h,nrmse 8h,nrmse,rmse 5h,rmse 6h,rmse 7h,rmse 8h,rmse,mae 5h,mae 6h,mae 7h,mae 8h,mae
model_name,interp,scal,opt,loss,lr,epochs,n_x,s,n_y,n_v,model,calendar_aware,weather_aware,trained_in,trained_with,t,vn,,,,,,,,,,,,,,,,
conv3d_stand_nearest_J0.0_nx04_ny04_sh05_mse_lr0.001_Adamax_ep100_BEN,nearest,stand,Adamax,mse,0.001,100,4,5,4,1,conv3d,False,False,limonero,NVIDIA GeForce RTX 2070,00:09,BEN,30,1.032,1.016,0.997,0.988,1.008,0.460,0.452,0.453,0.452,0.454,0.310,0.300,0.304,0.304,0.304
conv3d_stand_nearest_J0.0_nx04_ny04_sh05_mse_lr0.001_Adamax_ep100_CO,nearest,stand,Adamax,mse,0.001,100,4,5,4,1,conv3d,False,False,limonero,NVIDIA GeForce RTX 2070,00:24,CO,06,0.588,0.586,0.584,0.587,0.586,0.197,0.195,0.193,0.193,0.195,0.130,0.128,0.125,0.125,0.127
conv3d_stand_nearest_J0.0_nx04_ny04_sh05_mse_lr0.001_Adamax_ep100_EBE,nearest,stand,Adamax,mse,0.001,100,4,5,4,1,conv3d,False,False,limonero,NVIDIA GeForce RTX 2070,00:23,EBE,35,1.821,1.826,1.847,1.840,1.834,0.746,0.754,0.764,0.775,0.760,0.345,0.351,0.358,0.361,0.354
conv3d_stand_nearest_J0.0_nx04_ny04_sh05_mse_lr0.001_Adamax_ep100_NO,nearest,stand,Adamax,mse,0.001,100,4,5,4,1,conv3d,False,False,limonero,NVIDIA GeForce RTX 2070,00:28,NO,07,1.871,1.856,1.851,1.893,1.868,28.457,27.912,27.766,28.603,28.185,16.996,16.546,16.763,17.056,16.840
conv3d_stand_nearest_J0.0_nx04_ny04_sh05_mse_lr0.001_Adamax_ep100_NO2,nearest,stand,Adamax,mse,0.001,100,4,5,4,1,conv3d,False,False,limonero,NVIDIA GeForce RTX 2070,00:19,NO2,08,0.663,0.658,0.660,0.669,0.663,23.646,23.498,23.587,23.964,23.674,18.063,17.876,17.997,18.337,18.068
conv3d_stand_nearest_J0.0_nx04_ny04_sh05_mse_lr0.001_Adamax_ep100_NOx,nearest,stand,Adamax,mse,0.001,100,4,5,4,1,conv3d,False,False,limonero,NVIDIA GeForce RTX 2070,00:28,NOx,12,1.040,1.031,1.025,1.053,1.037,59.800,59.043,59.459,61.367,59.917,37.594,36.742,36.988,37.943,37.317
conv3d_stand_nearest_J0.0_nx04_ny04_sh05_mse_lr0.001_Adamax_ep100_O3,nearest,stand,Adamax,mse,0.001,100,4,5,4,1,conv3d,False,False,limonero,NVIDIA GeForce RTX 2070,00:29,O3,14,0.494,0.507,0.521,0.535,0.514,25.548,25.666,26.717,27.679,26.402,20.291,20.417,21.087,21.944,20.935
conv3d_stand_nearest_J0.0_nx04_ny04_sh05_mse_lr0.001_Adamax_ep100_PM10,nearest,stand,Adamax,mse,0.001,100,4,5,4,1,conv3d,False,False,limonero,NVIDIA GeForce RTX 2070,00:23,PM10,10,0.651,0.666,0.680,0.692,0.672,12.187,12.470,12.787,13.035,12.620,8.198,8.407,8.568,8.594,8.442
conv3d_stand_nearest_J0.0_nx04_ny04_sh05_mse_lr0.001_Adamax_ep100_PM25,nearest,stand,Adamax,mse,0.001,100,4,5,4,1,conv3d,False,False,limonero,NVIDIA GeForce RTX 2070,00:08,PM25,09,0.629,0.646,0.651,0.665,0.648,6.685,6.746,6.812,6.860,6.776,4.731,4.875,4.914,4.969,4.872
